In [8]:
# !pip install genomic_benchmarks
# !pip install omegaconf

In [1]:
# using Kernal PyTorch-2.0.1

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel #, AutoModelForMaskedLM

# import pickle
from tqdm import tqdm

# import pickle
# import re
import os
import csv
import warnings
warnings.filterwarnings('ignore')


%run hyena_utility.py

%run preprocess_utility.py

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

/scratch/local/48992548/ipykernel_1145272/1527746069.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/sunhuaikuan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

### Load Human Chrom Sequences from .fa File

In [2]:
fasta_file = "/blue/xiaofan/sunhuaikuan/gpn/examples/msa/genome.hg38rg.fa"
chrom_sequences = read_fasta(fasta_file)

In [3]:

# def Cur_Inference(model, tokenizer, max_length, device, sequence):
def Cur_Inference(model, sequence):  # model, tokenizer, max_length, device, 

    '''
    this selects which backbone to use, and grabs weights/ config from HF
    4 options:
      'hyenadna-tiny-1k-seqlen'   # fine-tune on colab ok
      'hyenadna-small-32k-seqlen'
      'hyenadna-medium-160k-seqlen'  # inference only on colab
      'hyenadna-medium-450k-seqlen'  # inference only on colab
      'hyenadna-large-1m-seqlen'  # inference only on colab
    '''

    # you only need to select which model to use here, we'll do the rest!
    # pretrained_model_name = 'hyenadna-small-32k-seqlen'

    # max_lengths = {
    #     'hyenadna-tiny-1k-seqlen': 1024,
    #     'hyenadna-small-32k-seqlen': 32768,
    #     'hyenadna-medium-160k-seqlen': 160000,
    #     'hyenadna-medium-450k-seqlen': 450000,  # T4 up to here
    #     'hyenadna-large-1m-seqlen': 1_000_000,  # only A100 (paid tier)
    # }


    #### Single embedding example ####

    # create a sample 450k long, prepare
    # sequence = 'ACTG' * int(max_length/4)
    tok_seq = tokenizer(sequence)
    tok_seq = tok_seq["input_ids"]  # grab ids

    # place on device, convert to tensor
    tok_seq = torch.LongTensor(tok_seq).unsqueeze(0)  # unsqueeze for batch dim
    tok_seq = tok_seq.to(device)

    # prep model and forward
    # model.to(device)
    # model.eval()
    with torch.inference_mode():
        embeddings = model(tok_seq)

    # cls_embedding = embeddings.last_hidden_state[:, 0, :]
    # cls_embedding = embeddings[:, 0, :]
    
    mean_embeddings = embeddings.mean(dim=1) # Mean across the sequence length dimension
    mean_embeddings = mean_embeddings.squeeze(0)  # This will change the shape to [256]

    
    # print(embeddings.shape)  # embeddings here!
    # return cls_embedding
    return mean_embeddings


In [4]:
# def Subsequence2Embedding(model, tokenizer, max_length, device, subsequence):
def Subsequence2Embedding(subsequence):
    # embeddings = My_Inference(model, tokenizer, max_length, device, subsequence)
    embeddings = Cur_Inference(model, subsequence)
    # print(embeddings.shape)
    # print (embeddings)
    # return embeddings[0,int(max_length/2),:]
    return embeddings  # embeddings[0,0,:]

# max_length

In [5]:
def append_rows_to_csv(csv_Filename, rows):
    with open(csv_Filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(rows)
        for row in rows:
            writer.writerow(row)

### Main Process

In [6]:
pretrained_model_name = 'hyenadna-small-32k-seqlen'
pretrained_model_name = 'hyenadna-medium-160k-seqlen'
# pretrained_model_name = 'hyenadna-medium-450k-seqlen'
# pretrained_model_name = 'hyenadna-large-1m-seqlen'
model, tokenizer, max_length =  get_model_tokenizer_maxlen(pretrained_model_name)
model.to(device)
model.eval()

Using device: cuda
Git LFS initialized.


Cloning into 'hyenadna-medium-160k-seqlen'...


Loaded pretrained weights ok!


HyenaDNAModel(
  (backbone): LMBackbone(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(16, 256)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): HyenaOperator(
          (dropout): Dropout(p=0.0, inplace=False)
          (in_proj): Linear(in_features=256, out_features=768, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
          (short_filter): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=(2,), groups=768)
          (filter_fn): HyenaFilter(
            (dropout): Dropout(p=0.0, inplace=False)
            (pos_emb): PositionalEmbedding()
            (implicit_filter): Sequential(
              (0): Linear(in_features=5, out_features=64, bias=True)
              (1): Sin()
              (2): Linear(in_features=64, out_features=64, bias=True)
              (3): Sin()
              (4): Linear(in_features=64, out_features=64, bias=True)
              (5): Sin()
              (6): Linear(in_features=

In [8]:
def get_subsequence(chrom_name, start_pos, length):
    
    # print(chrom_name, start_pos, length)
    if chrom_name in chrom_sequences:
        sequence = chrom_sequences[chrom_name]
        subsequence = sequence[start_pos:start_pos + length]
        return subsequence
    else:
        raise ValueError(f"Chromosome '{chrom_name}' not found in the FASTA file.")

In [9]:
%%time

comp = {'A':1, 'C':2, 'G':3, 'T':4}

max_length = 128

csv_Filename = './homo_sapiens_hyena_embedding.csv'
if os.path.exists(csv_Filename):
    os.remove(csv_Filename)

datafile_path = '/blue/xiaofan/chenyuanhan/data/Homo_sapiens.GRCh38.109.txt.gz'  
df = preprocess_home_sapiens_datafile(datafile_path)


df.loc[df['SIZE'] > max_length, 'END'] = df['START'] + max_length
df.loc[df['SIZE'] > max_length, 'SIZE'] = max_length
df.drop(columns=['END','TYPE','CLUSTER'], inplace=True)
# df


rows=[]
for index, row in df.iterrows():      
    chrom=row['CHROM']
    rowid=row['ROWID']
    # pos_start=row['POS']-round(max_length / 2)
    pos_start=row['START']

    if pos_start<=1:
        pos_start=1
    # ref=row['REF']
    # alt=row['ALT']        
    y=row['y']
    length = row['SIZE'] # max_length
    
    subsequence = get_subsequence(chrom, pos_start, length)
    if 'N' in subsequence:
        print("The character 'N' is present in the string.")
        
    embedding = Subsequence2Embedding(subsequence)
    # print(embedding.shape)
    # print (embedding)

    # feature=np.array(embedding_df.iloc[64])
    rows.append(np.append(embedding.cpu().numpy(),  [rowid, y]))  #chrom, length,  


    if index > 0 and (index % 2000) == 0:
        append_rows_to_csv(csv_Filename, rows)
        rows=[]
        print (f"index = {index} completed")
        
append_rows_to_csv(csv_Filename, rows)

print(f"Create File: "+csv_Filename)
    

index = 2000 completed
index = 4000 completed
index = 6000 completed
index = 8000 completed
index = 10000 completed
index = 12000 completed
index = 14000 completed
index = 16000 completed
index = 18000 completed
index = 20000 completed
index = 22000 completed
index = 24000 completed
index = 26000 completed
index = 28000 completed
index = 30000 completed
index = 32000 completed
index = 34000 completed
Create File: ./homo_sapiens_hyena_embedding.csv
CPU times: user 4min 29s, sys: 398 ms, total: 4min 29s
Wall time: 4min 42s


### Load CSV File

In [1]:
import pandas as pd

def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    column_names = [f'{i}' for i in range(1, df.shape[1]-1)]
    column_names.extend(['ROWID', 'y'])
    
    df.columns = column_names
    return df
df = load_embedding_file('./homo_sapiens_hyena_embedding.csv')
df

/scratch/local/48992548/ipykernel_1322225/1530850494.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,1,2,3,4,5,6,7,8,9,10,...,249,250,251,252,253,254,255,256,ROWID,y
0,-0.098259,-0.778371,-0.567632,-0.864359,-0.912805,-0.907104,1.634788,-0.941197,0.169670,0.114994,...,0.455128,-0.650662,-1.605736,-1.492585,1.182727,0.519747,0.673476,-1.080819,1.0,0.0
1,-0.225349,-0.772242,-0.580955,-1.056253,-1.319412,-0.937095,1.204180,-0.703993,-0.227651,0.182004,...,0.496497,-1.000214,-1.584138,-1.652526,1.257367,0.468274,0.278431,-0.962163,2.0,0.0
2,0.000942,-0.598621,-0.672390,-0.641442,-1.040335,-0.755668,1.274980,-0.978171,-0.042830,0.124320,...,0.494047,-0.849385,-1.569252,-1.409305,1.183825,0.358066,0.373570,-1.182730,3.0,0.0
3,-0.120756,-0.799536,-0.620240,-0.805628,-1.006669,-0.907013,1.548830,-0.936683,0.091012,0.218409,...,0.447258,-0.846772,-1.625529,-1.477122,1.251898,0.502077,0.407137,-1.082190,4.0,0.0
4,-0.087019,-0.803882,-0.531570,-0.888757,-1.351001,-0.758447,1.058772,-0.702686,-0.500163,0.162552,...,0.480105,-1.057028,-1.528152,-1.568092,1.297123,0.465277,0.306736,-0.968242,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34785,0.274643,-0.629633,-0.364290,-0.470770,-1.037111,-0.344028,0.983549,-0.706644,-0.610683,-0.003416,...,0.450930,-0.982729,-1.377394,-1.215747,1.303958,0.193141,0.440559,-0.997618,34832.0,6.0
34786,-0.120933,-0.795169,-0.486470,-0.896732,-0.729815,-0.907107,1.312369,-0.878555,-0.061940,0.434641,...,0.478314,-0.906501,-1.576636,-1.562356,1.228592,0.552661,0.454011,-0.952927,34833.0,6.0
34787,-0.214810,-0.918239,-0.492211,-0.741898,-0.633027,-1.155706,1.678414,-0.957490,0.181382,0.355271,...,0.178224,-0.777092,-1.570812,-1.401757,1.297300,0.631772,0.571384,-0.866733,34834.0,6.0
34788,-0.144414,-0.804913,-0.485522,-0.898138,-0.753247,-0.915142,1.296546,-0.867254,-0.063795,0.437570,...,0.474064,-0.915721,-1.579077,-1.566750,1.219721,0.559222,0.440362,-0.949460,34835.0,6.0
